In [1]:
import os
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy

from read_scope_data import read_trc_data, compare_trigger_times, get_trigger_time
from data_analysis_utils import Photons, calculate_stft
from plot_utils import plot_counts_per_bin, plot_photon_detection

%matplotlib qt

plt.rcParams.update({'font.size': 18})
plt.rcParams.update({'xtick.labelsize': 18, 'ytick.labelsize': 18})

In [2]:
ifn1 = r"E:\good_data\He3kA_B250G500G_pl0t20_uw17t27_P30\C1--E-ring-p30-z13-x200-Bdot--00020.trc"
ifn2 = r"E:\good_data\He3kA_B250G500G_pl0t20_uw17t27_P30\C2--E-ring-p30-z13-x200-Bdot--00020.trc"
ifn3 = r"E:\good_data\He3kA_B250G500G_pl0t20_uw17t27_P30\C3--E-ring-p30-z13-x200-Bdot--00020.trc"

By_P21, tarr_B = read_trc_data(ifn1)
Bx_P20, tarr_B = read_trc_data(ifn2)
By_P20, tarr_B = read_trc_data(ifn3)

Reading data...
Done
Reading data...
Done
Reading data...
Done


In [3]:
# Calculate STFT for each Bdot signal
freq_bins = 1000
overlap_fraction = 0.05
freq_min = 10e6  # 100 MHz
freq_max = 1000e6  # 800 MHz

# Calculate STFT for By_P21
freq1, stft_matrix1, stft_time1 = calculate_stft(
    time_array=tarr_B,
    data_arr=By_P21,
    freq_bins=freq_bins,
    overlap_fraction=overlap_fraction,
    window='hanning',
    freq_min=freq_min,
    freq_max=freq_max
)

# Calculate STFT for Bx_P20
freq2, stft_matrix2, stft_time2 = calculate_stft(
    time_array=tarr_B,
    data_arr=Bx_P20,
    freq_bins=freq_bins,
    overlap_fraction=overlap_fraction,
    window='hanning',
    freq_min=freq_min,
    freq_max=freq_max
)

# Calculate STFT for By_P20
freq3, stft_matrix3, stft_time3 = calculate_stft(
    time_array=tarr_B,
    data_arr=By_P20,
    freq_bins=freq_bins,
    overlap_fraction=overlap_fraction,
    window='hanning',
    freq_min=freq_min,
    freq_max=freq_max
)

# Create a figure with 3 subplots
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15), sharex=True)

# Plot STFT for By_P21 with log scale
im1 = ax1.imshow(stft_matrix1.T,
                 aspect='auto',
                 origin='lower',
                 extent=[stft_time1[0]*1e3, stft_time1[-1]*1e3, freq1[0]/1e6, freq1[-1]/1e6],
                 interpolation='None',
                 cmap='jet',
                 norm=colors.LogNorm(vmin=stft_matrix1.T[stft_matrix1.T > 0].min(), vmax=stft_matrix1.T.max()))
ax1.set_ylabel('Frequency (MHz)')
ax1.text(0.98, 0.95, 'By_P21 (x=8cm, y=0cm)', transform=ax1.transAxes, 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), 
         horizontalalignment='right', verticalalignment='top')
fig.colorbar(im1, ax=ax1, label='Magnitude')

# Plot STFT for Bx_P20 with log scale
im2 = ax2.imshow(stft_matrix2.T,
                 aspect='auto',
                 origin='lower',
                 extent=[stft_time2[0]*1e3, stft_time2[-1]*1e3, freq2[0]/1e6, freq2[-1]/1e6],
                 interpolation='None',
                 cmap='jet',
                 norm=colors.LogNorm(vmin=stft_matrix2.T[stft_matrix2.T > 0].min(), vmax=stft_matrix2.T.max()))
ax2.set_ylabel('Frequency (MHz)')
ax2.text(0.98, 0.95, 'Bx_P20 (x=0cm, y=0cm)', transform=ax2.transAxes, 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), 
         horizontalalignment='right', verticalalignment='top')
fig.colorbar(im2, ax=ax2, label='Magnitude')

# Plot STFT for By_P20 with log scale
im3 = ax3.imshow(stft_matrix3.T,
                 aspect='auto',
                 origin='lower',
                 extent=[stft_time3[0]*1e3, stft_time3[-1]*1e3, freq3[0]/1e6, freq3[-1]/1e6],
                 interpolation='None',
                 cmap='jet',
                 norm=colors.LogNorm(vmin=stft_matrix3.T[stft_matrix3.T > 0].min(), vmax=stft_matrix3.T.max()))
# ax3.set_xticks([10, 12, 14, 16, 18, 20])
ax3.set_xlabel('Time (ms)')
ax3.set_ylabel('Frequency (MHz)')
ax3.text(0.98, 0.95, 'By_P20 (x=0cm, y=0cm)', transform=ax3.transAxes, 
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8), 
         horizontalalignment='right', verticalalignment='top')
fig.colorbar(im3, ax=ax3, label='Magnitude')

# Adjust layout
plt.tight_layout()
plt.show()

Increased samples_per_fft to 10101 to accommodate frequency range
STFT calculated with 10102 samples per FFT window
Frequency range: 10.4 MHz to 999.8 MHz
Generated 5210 time points for 5210 FFT segments
Increased samples_per_fft to 10101 to accommodate frequency range
STFT calculated with 10102 samples per FFT window
Frequency range: 10.4 MHz to 999.8 MHz
Generated 5210 time points for 5210 FFT segments
Increased samples_per_fft to 10101 to accommodate frequency range
STFT calculated with 10102 samples per FFT window
Frequency range: 10.4 MHz to 999.8 MHz
Generated 5210 time points for 5210 FFT segments
